# Xarray demo

In a nutshell, [`xarray`](http://xarray.pydata.org/en/stable/) is n-dimensional `pandas`.

This means that instead of (or really, as well as) using positional integer indices, we can attach more meaningful indices to our data's dimensions. 

`xarray` calls these indices "coordinates".

### 1D data

In [ ]:
import xarray as xr

Let's make some data (e.g. a gamma-ray log) and some depths.

In [ ]:
data = [23, 34, 5, 45, 34, 56, 67, 45, 67] # GR values

In [ ]:
depths = [100,101,102,103,104,105,106,107,108] #  metres

Without `xarray`, it's a hassle to look up the data at a particular depth:

In [ ]:
data[depths.index(103)]

Instead, let's make an `xarray`:

In [ ]:
x = xr.DataArray(data,
                 name='GR',
                 coords=[depths],
                 dims=['Depth'],
                )

In [ ]:
x

In [ ]:
x[5]

In [ ]:
x.plot()

Notice how it labels everything for us!

This thing behaves more or less like a numpy array:

In [ ]:
x > 50

In [ ]:
x[3] * 123

What's really nice is that we can slice between indices without a key error:

In [ ]:
x.loc[104.5:107.67]

We can also interpolate values:

In [ ]:
x.interp(Depth=104.35)

And we can pass an array-like to this:

In [ ]:
x.interp(Depth=[104.2, 104.3, 104.4, 104.5, 104.6, 104.7])

### 2D data

Let's use made-up data again:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def f(x, y):
    return np.sin(x) + np.cos(y)

x_ = np.linspace(0, 2 * np.pi, 120)
y_ = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

z = f(x_, y_)
plt.imshow(z)

In [ ]:
z.shape

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
z[50:55, 50:55]

Now let's make some coordinates:

In [ ]:
x = np.arange(100) + 1000
y = np.arange(120) + 1000

In [ ]:
x

Now we can make the `xarray`:

In [ ]:
zx = xr.DataArray(z,
                  name='Elevation [m]',
                  coords=[x,y],
                  dims=['UTMx', 'UTMy'],
                 )

In [ ]:
zx.plot()

## 3D array

There are a couple of ways to go to higher dimensions:

- We could store new attributes of the 2D dataset.
- We could load a 3D volume of data with new coordinates.

First, let's add another dimension to the horizon - could be another attribute.

In [ ]:
zsq = np.sqrt(np.abs(z))

Make a 3D array with this new attribute in the first dimension:

In [ ]:
z2 = np.stack([z, zsq])

In [ ]:
attribs = ['Elev', 'SqrtElev']
z = xr.DataArray(z2,
                 name='Elevation [m]',
                 coords=[attribs, x, y],
                 dims=['Attribute', 'UTMx', 'UTMy'],
                )

In [ ]:
z

Plot one of the attributes...

In [ ]:
z[1].plot()

## `xarray` with seismic

#### [You will need to download this file.](https://geocomp.s3.amazonaws.com/data/F3_16-bit_int.sgy)

Put the file in any folder you can get to from this notebook, then continue here:

In [ ]:
import segyio

with segyio.open('F3_16-bit_int.sgy') as s:
    c = segyio.cube(s)

In [ ]:
c.shape

In [ ]:
il, xl, ts = c.shape

inlines = np.arange(il) + 1000
xlines = np.arange(xl) + 1000
tslices = 4 * np.arange(ts) / 1000

z = xr.DataArray(c,
                 name='Amplitude',
                 coords=[inlines, xlines, tslices],
                 dims=['Inline','Xline', 'Time'],
                )

This still works like a NumPy array:

In [ ]:
plt.imshow(z[:, :, 100], origin='lower', aspect=0.5)

But now we can do nice things like:

In [ ]:
z.loc[:, :, 0.5].plot()

In [ ]:
z.loc[1200].T.plot()
plt.gca().invert_yaxis()

For free, we got:

- Correct labeling of the axes and their ticks
- A correct title
- A divergent colourmap
- A labeled colourbar

## n-dimensional interpolation with `xarray`

Check out the end of [the SEG tutorial update notebook](https://github.com/seg/tutorials-2016/blob/master/1604_Function_of_interpolation/The_function_of_interpolation_with_xarray.ipynb) for a demo of using a horizon to interpolate into a 3D seismic volume. 

----

(c) Agile Scientific 2019